In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('paid_orders_mar10.csv')
ids=data.user_profile_id
out=data[ids.isin(ids[ids.duplicated()])].sort_values("user_profile_id")

In [3]:
out.head()

,user_profile_id,phone_number,name,order_id,order_date,payment_id,amount
28,00ec3ddf-f14e-4831-8917-05ccd5133b96,9631703450,Ashutosh,b11121f3-2d79-4c73-821c-0f622eb26f68,2021-01-05T02:26:34.000Z,cca406d7-93c1-429d-a452-5eebcea48db3,99
29,00ec3ddf-f14e-4831-8917-05ccd5133b96,9631703450,Ashutosh,22254518-0358-48f4-a465-c742a337a45f,2020-12-28T18:13:14.000Z,71f082fb-a017-419a-9635-526ef3be653c,99
50,01bc7695-97b4-42fe-9a75-fa2c33d9b01a,7978249766,NAMITA DASH,d30b46be-28f1-430b-b121-6c03b95b87dd,2020-12-04T14:43:38.000Z,9ac33980-3a4c-49d9-8353-4e193a9087a0,99
51,01bc7695-97b4-42fe-9a75-fa2c33d9b01a,7978249766,NAMITA DASH,bc32640f-fe6b-4689-9384-99b60654bc9a,2020-12-31T13:27:32.000Z,7b67db39-bfd4-4802-a81a-6285632d4472,99
68,0251e2ad-8db8-409c-8e79-002ea5b5d3ba,7503681589,Jyoti Singh,93e48f25-e36f-49f3-a8ea-1d47a052c921,2021-02-28T07:34:29.000Z,efa312b9-5666-4297-b129-c0a5d28f4cc7,99


In [4]:
def getdate(raw):
    d,_=raw.split('T')
    d=d.replace('-','')
    return int(d)
def backdate(date):
    date=str(date)
    return date[:4]+'-'+date[4:6]+'-'+date[6:]

In [5]:
getdate(out.order_date.iloc[0])

20210105

In [6]:
oc=out.copy()
oc.order_date=oc.order_date.apply(getdate)
oc.head()

,user_profile_id,phone_number,name,order_id,order_date,payment_id,amount
28,00ec3ddf-f14e-4831-8917-05ccd5133b96,9631703450,Ashutosh,b11121f3-2d79-4c73-821c-0f622eb26f68,20210105,cca406d7-93c1-429d-a452-5eebcea48db3,99
29,00ec3ddf-f14e-4831-8917-05ccd5133b96,9631703450,Ashutosh,22254518-0358-48f4-a465-c742a337a45f,20201228,71f082fb-a017-419a-9635-526ef3be653c,99
50,01bc7695-97b4-42fe-9a75-fa2c33d9b01a,7978249766,NAMITA DASH,d30b46be-28f1-430b-b121-6c03b95b87dd,20201204,9ac33980-3a4c-49d9-8353-4e193a9087a0,99
51,01bc7695-97b4-42fe-9a75-fa2c33d9b01a,7978249766,NAMITA DASH,bc32640f-fe6b-4689-9384-99b60654bc9a,20201231,7b67db39-bfd4-4802-a81a-6285632d4472,99
68,0251e2ad-8db8-409c-8e79-002ea5b5d3ba,7503681589,Jyoti Singh,93e48f25-e36f-49f3-a8ea-1d47a052c921,20210228,efa312b9-5666-4297-b129-c0a5d28f4cc7,99


In [7]:
groups=oc.groupby('name',group_keys=False)

In [8]:
new=groups.apply(lambda x: x.sort_values(by=['order_date']))

In [9]:
new.order_date=new.order_date.apply(backdate)

In [50]:
unique=new.pivot_table(index=['user_profile_id'],aggfunc='size')

In [59]:
unique=pd.DataFrame(unique,columns=['orders'])
unique['user_profile_id']=unique.index

In [11]:
len(new.user_profile_id.unique())

554

In [37]:
new_u=new.drop(['order_date','order_id','payment_id','amount'],axis=1).drop_duplicates()

In [55]:
new_u

,user_profile_id,phone_number,name
86,02d64cad-672a-4dc6-9678-f94715460cbd,8210525597,ABU NOMAN
973,1bced372-7b64-4123-8725-9afaed12d42e,9320121784,ALTAMASH
5595,a55b39b5-9a32-40b6-b875-7368e64280df,9609536185,AMIT SANTRA
4501,857042f2-e3ff-4542-b3fd-ac10e3cf3d68,9556452695,ANANT CHOUDHARY
6777,c893a9cd-be47-4edc-bd8e-17300c80ca84,8054051400,ANCHAL AGGARWAL
...,...,...,...
5850,abef8d46-d9e2-46f1-954f-f6e3a2d18919,9888046111,vipul batra
2677,4f6b4261-c3fa-411e-b21f-1be38cdb52f4,8806259026,virendra ingole
4515,85b0f1a6-1ed8-4d27-aefd-5f3cc44ddeea,7066611634,vishnu sharma
2042,3c9bd632-061d-43c1-babb-9a070808792b,8279862509,yashi sharma


In [64]:
unique=unique.reset_index(drop=True)

In [67]:
new_data=pd.merge(new_u,unique,on=['user_profile_id','user_profile_id'])

In [68]:
new_data

,user_profile_id,phone_number,name,orders
0,02d64cad-672a-4dc6-9678-f94715460cbd,8210525597,ABU NOMAN,2
1,1bced372-7b64-4123-8725-9afaed12d42e,9320121784,ALTAMASH,4
2,a55b39b5-9a32-40b6-b875-7368e64280df,9609536185,AMIT SANTRA,2
3,857042f2-e3ff-4542-b3fd-ac10e3cf3d68,9556452695,ANANT CHOUDHARY,9
4,c893a9cd-be47-4edc-bd8e-17300c80ca84,8054051400,ANCHAL AGGARWAL,2
...,...,...,...,...
549,abef8d46-d9e2-46f1-954f-f6e3a2d18919,9888046111,vipul batra,2
550,4f6b4261-c3fa-411e-b21f-1be38cdb52f4,8806259026,virendra ingole,3
551,85b0f1a6-1ed8-4d27-aefd-5f3cc44ddeea,7066611634,vishnu sharma,2
552,3c9bd632-061d-43c1-babb-9a070808792b,8279862509,yashi sharma,2
